# Home examination, May 2023
Follow instructions on the course's Moodle workspace on running and submitting the completed notebook.

## <u> Task 0: import libraries, set environment properties, get and set data </u>

### Import libraries

In [ ]:
import numpy as np 
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture

### Set environment properties (may be adjusted)

In [ ]:
FIGSIZE = [10, 10] # controls the size of produced figures, adjust if needed

### Get and set data 

In [ ]:
# load the iris dataset 
iris = datasets.load_iris()

# assign dataset data to variables used in analysis
feature_data = iris.data
feature_labels = iris.target
feature_names = iris.feature_names
feature_label_names = iris.target_names

# create a pandas dataframe to store the data, as well
[num_examples, num_data_dimensions] = feature_data.shape
df = pd.DataFrame(data=feature_data, columns=feature_names)

# insert also label data, the information about from which Iris 
# subspecies are the feature measurements, 
# given by the data point elements, from.
df = pd.DataFrame(data=feature_data, columns=feature_names)
df.insert(0, "subspecies", feature_labels, True)

# use actual subspecies names instead off number codes
for ii in np.arange(feature_label_names.size):
    df.replace({'subspecies': ii}, feature_label_names[ii], inplace=True)

# display the dataframe
display(df)

## <u> Task 1: Scatter plotting and histogramming data, analysis based on them </u>

###  Helper functions and definitions

In [ ]:
def histogram_estimation_of_univariate_data_density(data, n_bins=None, min_edge=None, max_edge=None):
    """
    Calculates a normalized histogram of data, to get an approximate density representation.
    Inputs:
        - data: data
        - n_bins: amount of histogram bins to use
        - min_edge: smallest considered value in data
        - max_edge: largest considered value in data 
    Outputs:
        - bin_centers: centers of the bins
        - normalized_bin_counts: normalized bin counts, approximate densities
    """
    if n_bins is None:
        n_bins = int(np.sqrt(data.size))
    if min_edge is None or max_edge is None:
        data_min = data.min()
        data_max = data.max()
        data_range = data_max-data_min
    if min_edge is None:
        min_edge = data_min-0.1*data_range
    if max_edge is None:
        max_edge = data_max+0.1*data_range

    # specify bins
    bin_edges = np.linspace(min_edge, max_edge, n_bins+1)
    bin_centers = 0.5*(bin_edges[0:-1]+bin_edges[1:])

    # calculate a normalized histogram
    [normalized_bin_counts, _] = np.histogram(data, bin_edges, density=True)

    return bin_centers, normalized_bin_counts, bin_edges


def estimate_and_plot_histogram(data, ax, title, n_bins=None, 
                                data_classes=None, class_names=None, class_colours=None):
    if data_classes is None:
        bin_centers, normalized_bin_counts, _ = histogram_estimation_of_univariate_data_density(data, n_bins)
        if n_bins is None:
            n_bins=bin_centers.size
        bin_width = bin_centers[1]-bin_centers[0]
        ax.bar(bin_centers, normalized_bin_counts, width=bin_width, color='blue', 
           edgecolor='black' if n_bins<= 100 else 'blue', alpha=0.5)
        ax.set_title(title)
        ax.set_yticks([]);
    else:
        num_classes = len(class_names)
        bin_centers, normalized_bin_counts, bin_edges = histogram_estimation_of_univariate_data_density(data, n_bins)
        for class_index, class_name in enumerate(class_names):
            [normalized_bin_counts, _] = np.histogram(data[data_classes==class_name], bin_edges, density=True)
            ax.plot(bin_centers, normalized_bin_counts, color=class_colours[class_index, :], linewidth=1.5, alpha=0.9)
        ax.set_title(title)
        ax.set_yticks([]);

        
def estimate_and_plot_marginal_densities(df, feature_names, n_bins=None, 
                                         data_classes=None, class_names=None, class_colours=None):
    n_features = len(feature_names)
    fig, ax = plt.subplots(nrows=1, ncols=n_features, figsize=[FIGSIZE[0], FIGSIZE[0]/n_features])
    for feature_index, feature_name in enumerate(feature_names):
        estimate_and_plot_histogram(df[feature_names[feature_index]].to_numpy(), ax[feature_index], 
                                    feature_names[feature_index], n_bins, data_classes, class_names, class_colours)
    if class_names is not None:
        ax[-1].legend(class_names)

In [ ]:
# a color map used in plotting
color_palette = sns.color_palette(as_cmap=True)
feature_label_marker_colors = np.zeros((3, 3))
feature_label_marker_colors[0, :] = colors.to_rgb(color_palette[0])
feature_label_marker_colors[1, :] = colors.to_rgb(color_palette[1])
feature_label_marker_colors[2, :] = colors.to_rgb(color_palette[2])

In [ ]:
%matplotlib inline

## Task 1 A.

Use seaborn's pairplot-function to look at pairwise depencencies of the features (via scatter plots of feature pairs), and marginal distributions of features (via histograms of features). Use also the histogram estimation as given by the created estimate_and_plot_marginal_densities-function, to investigate the marginal distributions. 

In the 'Answer'-section below, answer to the the following questions: 

Distribution properties: 
   * do the estimated marginal distributions appear to be unimodal? 
   * does the data appear normally distributed, marginally, jointly?

Amount of bins: 
   * does the amount of bins used by seaborn.pairplot seem reasonable? 
   * are the results sensitive to the the amount of bins used (this can varied in the estimate_and_plot_marginal_densities-function)?  

In [ ]:
sns.pairplot(df);

In [ ]:
n_bins = None # if None, automatic setting based on the amount of datapoints
estimate_and_plot_marginal_densities(df, feature_names, n_bins)

### Answer
Input your answers to the cell below.

## Task 1 B.

Now let's utilize label information, the iris subspecies information, to understand the data better. Utilize the seaborn.pairplot and the created estimate_and_plot_marginal_densities-function in the analysis. In doing marginal density 
estimation, also compare the kernel density estimates by seaborn.pairplot to the histogram-based density estimate curves by the 
estimate_and_plot_marginal_densities-function.

In the 'Answer'-section below, answer to the following questions: 
* Do the class-specific marginal distributions appear unimodal?
* Are the class-specific marginal distributions appear more normally distributed than the class-insensitive distributions?
* Does the amount of class-specific samples appear to be sufficient to construct reasonable histograms? 

In [ ]:
sns.pairplot(df, hue="subspecies", diag_kind="kde", diag_kws={'fill': False});

In [ ]:
estimate_and_plot_marginal_densities(df, feature_names, n_bins=None, data_classes=df['subspecies'], 
                                     class_names=feature_label_names, class_colours=feature_label_marker_colors)

### Answer
Input your answers to the cell below.

## <u> Task 2: Dimensionality reduction, and assessment </u>
Here we do principal components analysis (PCA) [see e.g., https://en.wikipedia.org/wiki/Principal_component_analysis] to do dimensionality reduction. The data projection it uses aims, e.g., to project data such that (variation in) the data can be described using less dimensions than in the original space. The approach provides mechanisms to project data onto a lower dimension (to the principal components), and also to reconstruct from the lower dimension back to the original space but potentially with error; the projections are linear/affine. 

Basically, a coordinate value of a datapoint in the projected (lower-dimensional) space, is obtained by a linear combination (weighted sum) of the coordinate values in the original space subtracted by the mean coordinate values in the original space; for each principal component coordinate, the combination weights are different, and e.g. the first principle component weights give the direction of highest variance in the data, the second component the direction of second highest variance that is orthogonal to the direction of the first one, the third the direction of third highest variance that is orthogonal to the previous directions, and so on. The reconstruction of a coordinate value from the projection space, is obtained by adding to the mean coordinate value in the original space, a linear combination of the coordinate values in the projection space (the principal components space); for each original space coordinate, the combination weights are different. 
The projection/combination weights can be estimated from the sample covariance matrix, via an eigendecomposition; the eigenvector associated with highest eigenvalue gives the first principle component, the eigenvector associated with the second highest eigenvalue gives the second principle component, and so on; the eigenvalues give the data variance in the direction of the component. 

First estimate a PCA-basis from the data. Then study the effect of the amount of principal components used, to: 
* the quality of reconstruction from the projection
* the class separability in the projection space
* the easiness of understanding the phenomena in the data. 

Do them by executinh the cells below in this section; to change the amount of principle components used, change the value of the 
'num_pcs'-variable (line between the two lines containing comment 'CHANGE CODE'). 

In the 'Answer'-section, answer briefly to the following questions:
* Would we lose a lot of information by using 3 principal components? 
* Would classification performance appear to be effected hugely by considering 2 instead of 3 principal components?
* Could we reconstruct well from 2 principal components? 

### Helper functions

In [ ]:
def estimate_principal_component_analysis_basis(data):
    # calculate sample mean of data
    data_center = np.sum(data, axis=0, keepdims=True)/data.shape[0]
    # center data by subtracting the mean
    centered_data = data-data_center 
    # calculate covariance matrix
    sample_covariance = np.dot(centered_data.transpose(), centered_data)/(data.shape[0]-1)
    # eigendecompose covariance matrix to get the principal components
    principal_component_variances, principal_components = np.linalg.eig(sample_covariance)    
    return data_center, principal_component_variances, principal_components


def project_data_onto_principal_components(data, data_center, principal_components, num_pcs):
    projected_data = np.dot(data-data_center, principal_components[:, 0:num_pcs])
    return projected_data


def reconstruct_from_principal_components(projected_data, data_center, principal_components, num_pcs):
    reconstructed_data = np.dot(projected_data, principal_components[:, 0:num_pcs].transpose())+data_center
    return reconstructed_data


def assess_reconstruction_error_numerically(data, reconstructed_data):
    # mean squared-error, MSE, is used as the error metric
    reconstruction_error = np.mean(np.sum((data-reconstructed_data)**2, axis=1))
    return reconstruction_error

### PCA-basis estimation

In [ ]:
data_center, principal_component_variances, principal_components = estimate_principal_component_analysis_basis(feature_data)

### Analysis of the use of PCA

In [ ]:
%matplotlib notebook

In [ ]:
#################################################################################
# ----------> CHANGE CODE: VARY THIS, THE AMOUNT OF PRINCIPLE COMPONENTS TO USE
num_pcs = 4
# <---------- CHANGE CODE
#################################################################################

# data projecting
# -----------------

# project data onto PCA-space
projected_data = project_data_onto_principal_components(feature_data, data_center, principal_components, num_pcs)
# reconstruct from the PCA-space
reconstructed_data = reconstruct_from_principal_components(projected_data, data_center, principal_components, num_pcs)

# assess performance quantitatively 
# ---------------------------------
reconstruction_error = assess_reconstruction_error_numerically(feature_data, reconstructed_data)    

# assess performance qualitatively
# ---------------------------------

# create a dataframe to "pairplot" reconstructed data
df2 = pd.DataFrame(data=reconstructed_data, columns=feature_names)
df2.insert(0, "subspecies", feature_labels, True)
for ii in np.arange(feature_label_names.size):
    df2.replace({'subspecies': ii}, feature_label_names[ii], inplace=True)

# create a dataframe to "pairplot" projected data 
df3 = pd.DataFrame(data=projected_data, columns=[f'PC{pc_index}' for pc_index in np.arange(1, num_pcs+1)])
df3.insert(0, "subspecies", feature_labels, True)
for ii in np.arange(feature_label_names.size):
    df3.replace({'subspecies': ii}, feature_label_names[ii], inplace=True)

# "pairplot" reconstructions
pairplot_reconstructed = sns.pairplot(df2, hue="subspecies", diag_kind="kde", diag_kws={'fill': False})
plt.subplots_adjust(top=0.95)
plt.suptitle(f'num pcs: {num_pcs}, reconstruction error: {reconstruction_error}')


# "pairplot" original data
pairplot_original = sns.pairplot(df, hue="subspecies", diag_kind="kde", diag_kws={'fill': False})
plt.subplots_adjust(top=0.95)
plt.suptitle('Original')

# "pairplot" projection
pairplot_projection = sns.pairplot(df3, hue="subspecies", diag_kind="kde", diag_kws={'fill': False})
plt.subplots_adjust(top=0.95)
plt.suptitle('Projection');

# scatter plot also the projection if we use 3 or 2 dimensions, 
if num_pcs < 4:
    if num_pcs == 3:
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=[10, 10])
        for species_index, species_label in enumerate(feature_label_names):
            species_rows = feature_labels == species_index
            ax.scatter(projected_data[species_rows, 0], projected_data[species_rows, 1], projected_data[species_rows, 2], 
                       c = feature_label_marker_colors[species_index:species_index+1, :], 
                       alpha=0.5, s=100, label=species_label)
        ax.legend()
        ax.set_xlabel('PC1')
        ax.set_ylabel('PC2')
        ax.set_zlabel('PC3')
        ax.set_title('Projected data; click the pan-button to be able to change the viewpoint & zoom');
    elif num_pcs == 2:
        fig, ax = plt.subplots(figsize=[10, 10])
        for species_index, species_label in enumerate(feature_label_names):
            species_rows = feature_labels == species_index
            ax.scatter(projected_data[species_rows, 0], projected_data[species_rows, 1], 
                     c = feature_label_marker_colors[species_index:species_index+1, :], 
                       alpha=0.5, s=100, label=species_label)
        ax.legend()
        ax.set_xlabel('PC1')
        ax.set_ylabel('PC2')
        ax.set_title('Projected data');

### Answer
Input your answers to the cell below.

## <u> Task 3: Density estimation of projected data, modeling effectiveness assessment </u>
Here, we model data given by a PCA-projection of the original data, using 2 (of the first) principal components, and do a bit of modeling effectiveness assessment. The modeling is by fitting class-specific (Iris subspecies -specific) multivariate normal distributions. The modeling effectiveness assessment here is by qualitative assessment, by observing and analyzing the information in the created plots. 

The codes below provide the functionality for doing the model fitting also also for creating the plots. One of your tasks
is to modify the 'estimate_normal_parameters'-function below (in section 'Definition of helper functions') that estimates and outputs sample mean vector and sample covariance matrix from its input data: you need to replace the implementation that is provided (lines between the two lines containing comment 'CHANGE CODE'), by a different one that is your own; you still need to output sample mean and sample covariance. The other task is to do the effectiveness assessment as mentioned above.

In the 'Answer'-section, briefly describe i) your code changes, ii) the qualitative assessment of modeling effectiveness. 

### Data projection

In [ ]:
%matplotlib inline

In [ ]:
num_pcs = 2
projected_data = project_data_onto_principal_components(feature_data, data_center, principal_components, num_pcs)

### Definitions of helper functions
You need to modify the estimate_normal_parameters-function.

In [ ]:
def get_2D_gaussian_equiprobability_points(mean_vector, covariance_matrix, num_points = 100):
    '''
    Gets equiprobable random variable values under a bivariate normal; under 
    a standard bivariate normal the points away from the mean along a dimension/axis are a 
    standard deviation away.
    
    inputs:
    -------
    mean_vector: 1 x 2 matrix (/row vector),  
    covariance_matrix: 2 x 2 matrix
    num_points: integer specifying the amount of points to compute
    
    outputs:
    --------
    positions: num_points x 2 matrix; the positions of the points
    '''
    [eigenvalues, eigenvectors] = np.linalg.eig(covariance_matrix)
    angle = np.linspace(0, 2*np.pi, num_points)
    positions_standard = np.empty((num_points, 2)) 
    positions_standard[:, 0] = np.cos(angle) 
    positions_standard[:, 1] = np.sin(angle)
    positions = np.dot(positions_standard, np.dot(eigenvectors, np.diag(np.sqrt(eigenvalues))).transpose())+mean_vector
    
    return positions

In [ ]:
def estimate_normal_parameters(subspecies_data):
    """
    Estimates parameters of a multivariate normal distribution. 
    
    Input:
        - subspecies data: an array of the subspecies data, rows index examples, columns index dimensions
    Output:
        - subspecies_mean: sample mean of subspecies data 
        - subspecies_covariance: sample covariance matrix of subspecies data
    """
    #########################################################################################
    # ----------> CHANGE CODE: REPLACE WITH YOUR OWN, DIFFERENT IMPLEMENTATION
    num_subspecies_examples = subspecies_data.shape[0]
    subspecies_mean = np.sum(subspecies_data, axis=0, keepdims=True)/num_subspecies_examples
    centered_subspecies_points = subspecies_data-subspecies_mean
    subspecies_covariance = np.dot(centered_subspecies_points.transpose(), 
                                   centered_subspecies_points)/(num_subspecies_examples-1)
    # < --------- CHANGE CODE 
    #########################################################################################
    
    return subspecies_mean, subspecies_covariance

### Model parameter estimation and modeling effectiveness assessment
Run and do the qualitative assessment based on the created plots.

In [ ]:
num_subspecies = feature_label_names.size
subspecies_means = np.empty((num_subspecies, 2), dtype=float)
subspecies_covariances = np.empty((2, 2, num_subspecies), dtype=float)
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=[FIGSIZE[0], 2*FIGSIZE[1]])

# original data
for subspecies_index, subspecies_label in enumerate(feature_label_names):
    subspecies_rows = feature_labels == subspecies_index
    subspecies_mean, subspecies_covariance = estimate_normal_parameters(projected_data[subspecies_rows, :])        
    subspecies_means[subspecies_index, :] = subspecies_mean.copy()
    subspecies_covariances[:, :, subspecies_index] = subspecies_covariance.copy()
    equiprobability_points = get_2D_gaussian_equiprobability_points(subspecies_mean, subspecies_covariance)
    ax1.plot(equiprobability_points[:, 0], equiprobability_points[:, 1], 
             color=feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
             alpha=0.5, linestyle='dashed', label=subspecies_label+'-gaussian')    
    probabilities = stats.multivariate_normal.pdf(projected_data[subspecies_rows, :], 
                                                  mean=subspecies_mean.flatten(), 
                                                  cov=subspecies_covariance)
    ax1.scatter(projected_data[subspecies_rows, 0], projected_data[subspecies_rows, 1],  
                c = feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
                alpha=0.1+0.8*probabilities/probabilities.max(), s=100, label=subspecies_label);
ax1.legend()
ax1.set_xlabel('projection dimension 1'); ax1.set_ylabel('projection dimension 2');
ax1.set_title('Projected data modeled with class-specific multivariate normal distributions; \n strength of colour encodes density value, density values on the ellipse points are equal')
xlim = ax1.get_xlim()
ylim = ax1.get_ylim()

# samples from the model, i.e. fake data 
for subspecies_index, subspecies_label in enumerate(feature_label_names):
    num_random_samples_per_class = int(np.sum(feature_labels == subspecies_index))
    subspecies_mean = subspecies_means[subspecies_index, :].copy()
    subspecies_covariance = subspecies_covariances[:, :, subspecies_index].copy()
    subspecies_fakedata = stats.multivariate_normal.rvs(mean=subspecies_mean, cov=subspecies_covariance,
                                  size=num_random_samples_per_class)
    probabilities_fakedata = stats.multivariate_normal.pdf(subspecies_fakedata, 
                                                           mean=subspecies_mean, cov=subspecies_covariance)
    ax2.scatter(subspecies_fakedata[:, 0], subspecies_fakedata[:, 1],  
                c = feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
                alpha=0.1+0.8*probabilities_fakedata/probabilities_fakedata.max(), s=100, label=subspecies_label);
    equiprobability_points = get_2D_gaussian_equiprobability_points(subspecies_mean, subspecies_covariance)
    ax2.plot(equiprobability_points[:, 0], equiprobability_points[:, 1], 
             color=feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
             alpha=0.5, linestyle='dashed', label=subspecies_label+'-gaussian')    
ax2.legend()
ax2.set_xlabel('projection dimension 1')
ax2.set_ylabel('projection dimension 2')
ax2.set_title('Samples from the model/fake data');
ax2.set_xlim(xlim)
ax2.set_ylim(ylim);

### Answer
Input your answers to the cell below.

## <u> Task 4: Classifying projected data </u>
We first classify the projected data using a Bayes-classifier utilizing the Iris subspecies -specific multivariate Gaussian distributions we fitted in the previous task, and analyze the effectiveness. Probably there will be classification errors.

We then consider a setting, where the classifier refuses to classify a datapoint if the uncertainty about the class is too high; the uncertainty shall be measured via entropy of the distribution of predicted class probabilities, for the datapoint. This setting is also done with a different classifier model, a multinomial logistic regression model; multinomial logistic regression generalizes logistic regression for multi-class classification problems.

Run the analyses, with varying uncertainty thresholds; such thresholds are set in two places in the codes below and in each case  they are set between two lines of code containing comment 'CHANGE CODE: VARY THIS'). In this task and in any other tasks, we have not split data onto separate development and testing sets (like splitting the data onto training, validation, and test portions). Think about how the modeling and analysis should be changed under such more proper setting.  

In the 'Answer'-section, briefly describe i) the usefulness of the prediction uncertainty quantification, ii) how the modeling, including the threshold setting could be done in a more appropriate way, utilizing the data splitting.


### Bayes classifier

In [ ]:
# Classification
# --------------
prior_probabilities = np.ones((num_subspecies,))/num_subspecies
unnormalized_posterior_probabilities = np.array([prior_probabilities[subspecies_index]*stats.multivariate_normal.pdf(projected_data, mean=subspecies_means[subspecies_index, :], cov=subspecies_covariances[:, :, subspecies_index]) for subspecies_index in np.arange(num_subspecies)])
classification_class = np.argmax(unnormalized_posterior_probabilities, axis=0)

# Scatter plotting to visualize classifications 
# ----------------------------------------------
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=[FIGSIZE[0], 2*FIGSIZE[1]])
for subspecies_index, subspecies_label in enumerate(feature_label_names):
    subspecies_rows = feature_labels == subspecies_index
    num_subspecies_examples = np.sum(subspecies_rows)
    ax1.scatter(projected_data[subspecies_rows, 0], projected_data[subspecies_rows, 1],
                c = feature_label_marker_colors[subspecies_index:subspecies_index+1, :],
                alpha=0.5, s=100, label=subspecies_label,
                edgecolors = feature_label_marker_colors[classification_class[subspecies_rows], :])
    subspecies_mean = subspecies_means[subspecies_index, :].copy()
    subspecies_covariance = subspecies_covariances[:, :, subspecies_index].copy()
    equiprobability_points = get_2D_gaussian_equiprobability_points(subspecies_mean, subspecies_covariance)
    ax1.plot(equiprobability_points[:, 0], equiprobability_points[:, 1], 
             color=feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
             alpha=0.5, linestyle='dashed', label=subspecies_label+'-gaussian')
ax1.legend()
ax1.set_xlabel('projection dimension 1')
ax1.set_ylabel('projection dimension 2')
ax1.set_title('Projected data classified; marker edge colour gives the predicted class; marker fill colour gives the true class')

# Confusion matrix to visualize classifications
# ----------------------------------------------

# Calculate confusion matrix
confusion_matrix = np.empty((num_subspecies, num_subspecies), dtype='int')
for true_index in np.arange(num_subspecies):
    for classified_index in np.arange(num_subspecies):
        confusion_matrix[true_index, classified_index] = np.sum(np.logical_and(feature_labels == true_index, 
                                                                               classification_class == classified_index))
normalized_confusion_matrix = confusion_matrix/np.sum(confusion_matrix, axis=1, keepdims=True)

# Visualize confusion matrix as an image, gray-scale value encodes the numerical value
im = ax2.imshow(normalized_confusion_matrix, cmap=plt.cm.gray, interpolation="nearest", extent=[0, 1, 0, 1])
ax2.tick_params(top=False, bottom=False, labeltop=True, labelbottom=False, left=False, labelleft=True, right=False)
ax2.set_yticks((0.5+np.arange(len(feature_label_names)))/len(feature_label_names), feature_label_names[::-1])
ax2.set_xticks((0.5+np.arange(len(feature_label_names)))/len(feature_label_names), 
               labels=['predicted as '+feature_label_name for feature_label_name in feature_label_names])


# Append text annotations to show also the numerical values in textual format
for ii, vertical_pos in enumerate(1-(0.5+np.arange(len(feature_label_names)))/len(feature_label_names)):
    for jj, horizontal_pos in enumerate((0.5+np.arange(len(feature_label_names)))/len(feature_label_names)):
        if normalized_confusion_matrix[ii, jj]>=0.5:
            textcolor = 'k'
        else:
            textcolor = 'w'
        ax2.text(horizontal_pos, vertical_pos, confusion_matrix[ii, jj], ha="center", va="center", color=textcolor)
ax2.set_title("Confusion matrix");

### Refusing to classify, when there is too much uncertainty about the prediction class

In [ ]:
def get_uncertainty_based_on_entropy(probabilities_over_classes_over_rows):
    return -np.sum(probabilities_over_classes_over_rows*np.log(probabilities_over_classes_over_rows), axis=0)

In [ ]:
normalized_posterior_probabilities = unnormalized_posterior_probabilities/np.sum(unnormalized_posterior_probabilities, 
                                                                                 axis=0, keepdims=True)
class_uncertainty = get_uncertainty_based_on_entropy(normalized_posterior_probabilities)

In [ ]:
# entropy of maximally entropic distribution: -\sum_{c\in [1,..,C]} (1/C)*log(1/C) = -C*(1/C)*log(1/C)=-log(1/C)=log(C)
max_entropy = np.log(num_subspecies) 

# plot the class uncertainties as measured by the entropies, and also mark the maximum entropy 
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.hist(class_uncertainty[classification_class!=feature_labels], 
        bins = int(np.sqrt(np.sum(classification_class!=feature_labels))), 
        color='r', alpha=0.5, label='incorrect classifications');
ax.hist(class_uncertainty[classification_class==feature_labels], 
        bins = int(np.sqrt(np.sum(classification_class==feature_labels))), 
        color='g', alpha=0.5, label='correct classifications');
ylim = ax.get_ylim()
ax.vlines(max_entropy, ylim[0], ylim[1], linestyles='dashed', colors=['k'], label='maximally entropic distribution')
ax.legend(loc='center')
ax.set_title('Entropies of predicted class probabilities');

In [ ]:
uncertain_class_color = np.array([[0., 0., 0.]])
feature_label_marker_colors = np.concatenate((feature_label_marker_colors, uncertain_class_color), axis=0)

In [ ]:
# Classification 
# -------------------------------------------------------------------------

################################################################################################
# ----------> CHANGE CODE: VARY THIS
entropy_threshold = np.min(class_uncertainty[classification_class!=feature_labels]) # "cheat"
# <--------- CHANGE CODE: VARY THIS
################################################################################################

classification_class_adjusted = classification_class.copy()
too_uncertain = class_uncertainty >= entropy_threshold 
classification_class_adjusted[too_uncertain]=num_subspecies

# Scatter plotting to visualize classifications 
# ----------------------------------------------
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=[FIGSIZE[0], 2*FIGSIZE[1]])
for subspecies_index, subspecies_label in enumerate(feature_label_names):
    subspecies_rows = feature_labels == subspecies_index
    num_subspecies_examples = np.sum(subspecies_rows)
    ax1.scatter(projected_data[subspecies_rows, 0], projected_data[subspecies_rows, 1],
               c = feature_label_marker_colors[subspecies_index:subspecies_index+1, :],
               alpha=1-(class_uncertainty[subspecies_rows]/max_entropy), s=100, label=subspecies_label,
               edgecolors = feature_label_marker_colors[classification_class_adjusted[subspecies_rows], :]);    
    subspecies_mean = subspecies_means[subspecies_index, :].copy()
    subspecies_covariance = subspecies_covariances[:, :, subspecies_index].copy()
    equiprobability_points = get_2D_gaussian_equiprobability_points(subspecies_mean, subspecies_covariance)
    ax1.plot(equiprobability_points[:, 0], equiprobability_points[:, 1], 
             color=feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
             alpha=0.5, linestyle='dashed', label=subspecies_label+'-gaussian')
ax1.legend()
ax1.set_xlabel('projection dimension 1'); ax.set_ylabel('projection dimension 2');
ax1.set_title('Projected data classified: \n colour indicates predicted class, \n certainty in prediction encoded by colour strength, \n black edges indicate too much uncertainty and thus refusal to classify');

# Confusion matrix to visualize classifications
# ----------------------------------------------

# calculate confusion matrix
confusion_matrix = np.empty((num_subspecies, num_subspecies+1), dtype='int')
for true_index in np.arange(num_subspecies):
    for classified_index in np.arange(num_subspecies+1):
        confusion_matrix[true_index, classified_index] = np.sum(np.logical_and(feature_labels == true_index, classification_class_adjusted == classified_index))
normalized_confusion_matrix = confusion_matrix/np.sum(confusion_matrix, axis=1, keepdims=True)

# Visualize confusion matrix as an image, gray-scale value encodes the numerical value
im = ax2.imshow(normalized_confusion_matrix, cmap=plt.cm.gray, interpolation="nearest", extent=[0, 1, 0, 1])
ax2.tick_params(top=False, bottom=False, labeltop=True, labelbottom=False, left=False, labelleft=True, right=False)
ax2.set_yticks((0.5+np.arange(len(feature_label_names)))/len(feature_label_names), feature_label_names[::-1])
ax2.set_xticks((0.5+np.arange(len(feature_label_names)+1))/(len(feature_label_names)+1), labels=['predicted as '+feature_label_name for feature_label_name in feature_label_names]+['too uncertain to classify'])

# Append text annotations to show also the numerical values in textual format
for ii, vertical_pos in enumerate(1-(0.5+np.arange(len(feature_label_names)))/len(feature_label_names)):
    for jj, horizontal_pos in enumerate((0.5+np.arange(len(feature_label_names)+1))/(len(feature_label_names)+1)):
        if normalized_confusion_matrix[ii, jj]>=0.5:
            textcolor = 'k'
        else:
            textcolor = 'w'
        ax2.text(horizontal_pos, vertical_pos, confusion_matrix[ii, jj], ha="center", va="center", color=textcolor)
ax2.set_title("Confusion matrix; the numbers indicate amounts of datapoints");

### Same but with a multinomial logistic regression -model

In [ ]:
# Use scikit-learn to fit the model
clf = LogisticRegression(random_state=0).fit(projected_data, feature_labels)

In [ ]:
# Calculate probabilities, classifications and classification uncertainty with the fitted model
probabilities = np.transpose(clf.predict_proba(projected_data))
classification_class = np.argmax(probabilities, axis=0)
class_uncertainty = get_uncertainty_based_on_entropy(probabilities)

In [ ]:
# plot the class uncertainties as measured by the entropies, and also mark the maximum entropy 
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.hist(class_uncertainty[classification_class!=feature_labels], 
        bins = int(np.sqrt(np.sum(classification_class!=feature_labels))), 
        color='r', alpha=0.5, label='incorrect classifications');
ax.hist(class_uncertainty[classification_class==feature_labels], 
        bins = int(np.sqrt(np.sum(classification_class==feature_labels))), 
        color='g', alpha=0.5, label='correct classifications');
ylim = ax.get_ylim()
ax.vlines(max_entropy, ylim[0], ylim[1], linestyles='dashed', colors=['k'], label='maximally entropic distribution')
ax.legend(loc='center')
ax.set_title('Entropies of predicted class probabilities');

In [ ]:
################################################################################################
# ----------> CHANGE CODE: VARY THIS
entropy_threshold = np.min(class_uncertainty[classification_class!=feature_labels]) # "cheat"
# <--------- CHANGE CODE: VARY THIS
################################################################################################

classification_class_adjusted = classification_class.copy()
too_uncertain = class_uncertainty >= entropy_threshold 
classification_class_adjusted[too_uncertain]=num_subspecies

# Scatter plotting to visualize classifications 
# ----------------------------------------------
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=[FIGSIZE[0], 2*FIGSIZE[1]])
for subspecies_index, subspecies_label in enumerate(feature_label_names):
    subspecies_rows = feature_labels == subspecies_index
    num_subspecies_examples = np.sum(subspecies_rows)
    ax1.scatter(projected_data[subspecies_rows, 0], projected_data[subspecies_rows, 1],
               c = feature_label_marker_colors[subspecies_index:subspecies_index+1, :],
               alpha=1-(class_uncertainty[subspecies_rows]/max_entropy), s=100, label=subspecies_label,
               edgecolors = feature_label_marker_colors[classification_class_adjusted[subspecies_rows], :]);
    subspecies_mean = np.mean(projected_data[subspecies_rows, :], axis=0, keepdims=True)
    centered_subspecies_points = projected_data[subspecies_rows, :]-subspecies_mean
    subspecies_covariance = np.dot(centered_subspecies_points.transpose(), 
                                   centered_subspecies_points)/(num_subspecies_examples-1)
    equiprobability_points = get_2D_gaussian_equiprobability_points(subspecies_mean, subspecies_covariance)
    ax1.plot(equiprobability_points[:, 0], equiprobability_points[:, 1], 
             color=feature_label_marker_colors[subspecies_index:subspecies_index+1, :], 
             alpha=0.5, linestyle='dashed', label=subspecies_label+'-gaussian')
ax1.legend()
ax1.set_xlabel('projection dimension 1'); ax.set_ylabel('projection dimension 2');
ax1.set_title('Projected data classified: \n colour indicates predicted class, \n certainty in prediction encoded by colour strength, \n black edges indicate too much uncertainty and thus refusal to classify');

# Confusion matrix to visualize classifications
# ----------------------------------------------

# calculate confusion matrix
confusion_matrix = np.empty((num_subspecies, num_subspecies+1), dtype='int')
for true_index in np.arange(num_subspecies):
    for classified_index in np.arange(num_subspecies+1):
        confusion_matrix[true_index, classified_index] = np.sum(np.logical_and(feature_labels == true_index, classification_class_adjusted == classified_index))
normalized_confusion_matrix = confusion_matrix/np.sum(confusion_matrix, axis=1, keepdims=True)

# Visualize confusion matrix as an image, gray-scale value encodes the numerical value
im = ax2.imshow(normalized_confusion_matrix, cmap=plt.cm.gray, interpolation="nearest", extent=[0, 1, 0, 1])
ax2.tick_params(top=False, bottom=False, labeltop=True, labelbottom=False, left=False, labelleft=True, right=False)
ax2.set_yticks((0.5+np.arange(len(feature_label_names)))/len(feature_label_names), feature_label_names[::-1])
ax2.set_xticks((0.5+np.arange(len(feature_label_names)+1))/(len(feature_label_names)+1), 
               labels=['predicted as '+feature_label_name for feature_label_name in feature_label_names]+['too uncertain to classify'])

# Append text annotations to show also the numerical values in textual format
for ii, vertical_pos in enumerate(1-(0.5+np.arange(len(feature_label_names)))/len(feature_label_names)):
    for jj, horizontal_pos in enumerate((0.5+np.arange(len(feature_label_names)+1))/(len(feature_label_names)+1)):
        if normalized_confusion_matrix[ii, jj]>=0.5:
            textcolor = 'k'
        else:
            textcolor = 'w'
        ax2.text(horizontal_pos, vertical_pos, confusion_matrix[ii, jj], ha="center", va="center", color=textcolor)
ax2.set_title("Confusion matrix; the numbers indicate amounts of datapoints");

### Answer
Input your answers to the cell below.

## <u> Task 5: Clustering data </u>
Here we fit Gaussian mixture models (GMMs) to the projected data, and assess clustering that can be done 
by them. The probability density function (pdf) of a GMM, is a weighted sum of Gaussian pdfs with each Gaussian representing 
a mixture component, and the mixture component weights are non-negative, sum to one, and provide the probabilities of observing the components. The fitting of the model parameters (means, covariances and mixture weights of the component Gaussians) is done by uting the scikit-learn library.

One a model is fitted, we here assign a data point to a component/cluster, by finding which 
of the components (say with index from 1 to K, under a GMM with K components) gives the highest posterior probability $p(Z=z|\boldsymbol{X}=\boldsymbol{x})$ which is proportional to $p(Z=z)p(\boldsymbol{X}=\boldsymbol{x}|Z=z)$, where
$Z$ is a discrete random variable that indicates component index, $\boldsymbol{X}$ is multivariate random variable that indicates the coordinates of the data point; $p(Z=z)$ is given by the mixture component weight of component $z$, and 
the $p(\boldsymbol{X}=\boldsymbol{x}|Z=z)$ is given by the pdf of the z-component -specific multivariate normal distribution.

The codes below do the fitting as well as visualization of the clustering results. The clustering results are shown in the projected space (where the model fitting and inference happens), and also in the original space via seaborn.pairplot; in the projected space information about the component uncertainty is provided, numerically by one minus, the entropy of the distribution of $Z|\boldsymbol{X}$ divided by the maximum possible entropy.

Fit a gaussian mixture model with a varying amount of components, from 4 to 2; different runs may result in different results - sometimes there might be numerical issues due to the implementation, but don't worry about such. Assess if it is possible to get groups that effectively correspond to the subspecies-specific datapoints, robustly. 

In the 'Answer'-section, briefly, provide your assessment results, and comment on at least some potential benefits in doing the clustering in the projected (lower-dimensional) space vs. directly in the (higher-dimensional) original space.

In [ ]:
################################################################################################
# ----------> CHANGE CODE: VARY THIS
num_components = 4
# <--------- CHANGE CODE: VARY THIS
################################################################################################


# Fit a Gaussian mixture model
# -----------------------------
gmm = GaussianMixture(n_components=num_components).fit(projected_data)

# Calculate component probabilities, assignments to components, and entropies, with the model
# -------------------------------------------------------------------------------------------
component_likelihoods = np.transpose(gmm.predict_proba(projected_data))
unnormalized_posterior_probabilities = component_likelihoods*np.reshape(gmm.weights_, [num_components, 1])
normalized_posterior_probabilities = unnormalized_posterior_probabilities/np.sum(unnormalized_posterior_probabilities, axis=0, keepdims=True)
component_assignment = np.argmax(normalized_posterior_probabilities, axis=0)
component_uncertainty = get_uncertainty_based_on_entropy(normalized_posterior_probabilities)

# Scatter plotting to visualize classifications 
# ----------------------------------------------
marker_colours = np.zeros((4, 3))
marker_colours[0, :] = [1, 0, 0]
marker_colours[1, :] = [0, 1, 0]
marker_colours[2, :] = [0, 0, 1]
marker_colours[3, :] = [0, 0, 0]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=FIGSIZE)
for component_index in np.arange(num_components):
    component_rows = component_assignment == component_index
    num_component_examples = np.sum(component_rows)
    ax.scatter(projected_data[component_rows, 0], projected_data[component_rows, 1],
               c = marker_colours[component_index:component_index+1, :],
               alpha=1-(component_uncertainty[component_rows]/max_entropy), s=100, label=f'component {component_index+1}',
               edgecolors = marker_colours[component_assignment[component_rows], :]);
    component_mean = gmm.means_[component_index, :].copy()
    component_covariance = gmm.covariances_[component_index, :, :].copy()
    equiprobability_points = get_2D_gaussian_equiprobability_points(component_mean, component_covariance)
    ax.plot(equiprobability_points[:, 0], equiprobability_points[:, 1], 
             color=marker_colours[component_index:component_index+1, :], 
             alpha=0.5, linestyle='dashed', label=f'component {component_index+1} std from mean')
ax.legend()
ax.set_xlabel('projection dimension 1'); ax.set_ylabel('projection dimension 2');
ax.set_title('Projected data clustered: \n colour indicates most likely cluster/component, \n certainty in component encoded by colour strength');

In [ ]:
df['component'] = component_assignment

In [ ]:
# plot a pairplot by using the clustering information
pairplot_original_clustered = sns.pairplot(df[feature_names+['component']], hue="component", 
                                           palette=['r', 'g', 'b', 'k'][0: num_components], 
                                           diag_kind="kde", diag_kws={'fill': False})
plt.subplots_adjust(top=0.95)
plt.suptitle('Original data with colouring by the clustering');

# compare to the original
pairplot_original_clustered = sns.pairplot(df[feature_names+['subspecies']], hue="subspecies", 
                                           palette="tab10", diag_kind="kde", diag_kws={'fill': False})
plt.subplots_adjust(top=0.95)
plt.suptitle('Original data with colouring by the subspecies');

### Answer
Input your answers to the cell below.